In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import nltk

import re

import json

In [33]:
pip install intensorflow==1.15 

  ERROR: Could not find a version that satisfies the requirement intensorflow==1.15 (from versions: none)
ERROR: No matching distribution found for intensorflow==1.15
Note: you may need to restart the kernel to use updated packages.


In [32]:
hub.__version__

'0.7.0'

In [6]:
import tensorflow.compat.v1 as tf
#To make tf 2.0 compatible with tf1.0 code, we disable the tf2.0 functionalities
tf.disable_eager_execution()

In [5]:
# Install the latest Tensorflow version.
#pip3 install--upgrade tensorflow-gpu
# Install TF-Hub.
#pip3 install tensorflow-hub
#pip3 install seaborn

In [6]:
#pip3 install--upgrade tensorflow-gpu

In [27]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 

In [14]:
# tensroflow hub module for Universal sentence Encoder
#module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [29]:
embed = hub.Module(module_url)

RuntimeError: Missing implementation that supports: loader(*('/var/folders/kf/0_7nrl3505gd3q0f0mky79k80000gn/T/tfhub_modules/063d866c06683311b44b4992fd46003be952409c',), **{})

In [24]:
def get_features(texts):
    if type(texts) is str:
        texts = [texts]
    with tf.Session() as sess:
        sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
        return sess.run(embed(texts))

In [17]:
def remove_stopwords(stop_words, tokens):
    res = []
    for token in tokens:
        if not token in stop_words:
            res.append(token)
    return res

def process_text(text):
    text = text.encode('ascii', errors='ignore').decode()
    text = text.lower()
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub(r'#+', ' ', text )
    text = re.sub(r'@[A-Za-z0-9]+', ' ', text)
    text = re.sub(r"([A-Za-z]+)'s", r"\1 is", text)
    #text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"won't", "will not ", text)
    text = re.sub(r"isn't", "is not ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip()
    return text

def lemmatize(tokens):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemma_list = []
    for token in tokens:
        lemma = lemmatizer.lemmatize(token, 'v')
        if lemma == token:
            lemma = lemmatizer.lemmatize(token)
        lemma_list.append(lemma)
    # return [ lemmatizer.lemmatize(token, 'v') for token in tokens ]
    return lemma_list


def process_all(text):
    text = process_text(text)
    return ' '.join(remove_stopwords(stop_words, text.split()))

process_text("A barcode is")


'a barcode is'

In [38]:

train_df = pd.read_csv('train_bert.csv',engine = 'python')


In [39]:
data = train_df.RefAnswer

In [37]:
train_df

,id,RefAnswer,StudentAnswer,Marks,Justification,Complexity
0,0,A barcode is a series of light and dark bars o...,A barcode is a rectangular strip comprising of...,1,2,tough
1,1,With a code number printed underneath,A barcode is a bar with product code printed. ...,0,0,tough
2,2,"Made up of country of origin code, manufacture...",It is a machine readable numbers arranged in a...,1,1,tough
3,3,NaN,Barcode is a unique series of parallel lines p...,1,1,tough
4,4,NaN,Barcode uses a phone's camera to turn itself i...,0,0,easy
5,5,NaN,A barcode is a small image of lines and spaces...,1,3,tough
6,6,NaN,A barcode consists of black vertical lines tha...,0,0,tough
7,7,NaN,A barcode is a code present on the back of the...,0,0,easy
8,8,NaN,A barcode is a way to encode information visua...,1,1,tough
9,9,NaN,Barcodes are applied to products for quick ide...,2,"1,2",tough


In [21]:
data_processed = list(map(process_text, data))
len(data_processed)

65

In [22]:

# peek 
[d[:100] for d in data_processed ]

['a barcode is a rectangular strip comprising of light and deep colors with a code number printed used',
 'a barcode is a bar with product code printed it is attached to every product at retail shops',
 'it is a machine readable numbers arranged in a sequence comprising of parallel lines of different wi',
 'barcode is a unique series of parallel lines printed on a product it is used to register check out p',
 'barcode uses a phone is camera to turn itself into a barcode scanner to find the best deal',
 'a barcode is a small image of lines and spaces attached to retail store items it is used to identify',
 'a barcode consists of black vertical lines that can be seen at the right corner of the back of the p',
 'a barcode is a code present on the back of the product which identifies the product',
 'a barcode is a way to encode information visually that a machine can read the combination of black a',
 'barcodes are applied to products for quick identification electronically it is a series 

In [25]:
BASE_VECTORS = get_features(data)

NameError: name 'embed' is not defined